In [1]:
import numpy as np
import scipy

from qibo import gates
from qibo.transpiler.unitary_decompositions import two_qubit_decomposition
from qibo.models.qdp.quantum_dynamic_programming import (
    AbstractQuantumDynamicProgramming,
    QDPSequentialInstruction,
    QDPMeasurementEmulation,
    QDPMeasurementReset
    )
from qibo.models.dbi.double_bracket import (
    DoubleBracketGeneratorType,
    DoubleBracketIteration,
)

In [ ]:
class QDP2QubitsSequentialInstruction(AbstractQuantumDynamicProgramming):

    def memory_call_circuit(self, num_instruction_qubits_per_query):
        """
        Executes the memory call circuit. Every instruction qubit is used once then discarded.

        Args:
            num_instruction_qubits_per_query (int): Number of instruction qubits per query.
        """
        current_instruction_index = self.instruction_index(
            self.id_current_instruction_reg
        )
        self.list_id_current_instruction_reg = self.list_id_instruction_reg[
            current_instruction_index : self.M * num_instruction_qubits_per_query
            + current_instruction_index
        ]
        self.instruction_qubits_initialization()
        # 2 quibits
        for _register in self.list_id_current_instruction_reg[1::2]:
            self.memory_usage_query_circuit()
            self.trace_one_instruction_qubit(_register)
            self.trace_one_instruction_qubit(_register+1)
            if self.instruction_index(_register) + 2 < len(
                list(self.list_id_current_instruction_reg)
            ):
                self.increment_current_instruction_register()
            self.instruction_reg_delegation()

In [ ]:
class ObliviousSchmidtDecomposition(QDP2QubitsSequentialInstruction):
    """
    Subclass of AbstractQuantumDynamicProgramming for density matrix exponentiation,
    where we attempt to instruct the work qubit to do an X rotation, using SWAP gate.

    Args:
        rho_A : first qubit of the bipartite instruction state
        t: total duration
        num_iter: number of Trotterization step
        num_work_qubits (int): Number of work qubits.
        num_instruction_qubits (int): Number of instruction qubits.
        number_muq_per_call (int): Number of memory units per call.
    """

    def __init__(
        self, rho_A, s_n, num_iter, num_work_qubits, num_instruction_qubits, number_muq_per_call
    ):
        super().__init__(
            num_work_qubits, num_instruction_qubits, number_muq_per_call, circuit=None
        )
        self.rho_A = rho_A
        self.s_n = s_n
        self.num_iter = num_iter

    def memory_usage_query_circuit(self):
        """Defines the memory usage query circuit."""
        D = DoubleBracketIteration(self.rho_A).diagonal_h_matrix
        operator_0 = 
        operator_1 = 
        N = 1j * self.s_n * (D[0][0]*operator_0 + D[1][1]*operator_1)
        r = self.t/self.num_iter
        N_unitary = scipy.linalg.expm(
            -1j * N * r
        )
        for decomposed_gate in two_qubit_decomposition(
            self.id_current_work_reg,
            self.id_current_instruction_reg,
            unitary=N_unitary,
        ):
            self.c.add(decomposed_gate)
        

    def instruction_qubits_initialization(self):
        """Initializes the instruction qubits."""
        for instruction_qubit in self.list_id_current_instruction_reg[1::2]:
            self.c.add(gates.X(instruction_qubit)) #rho_A
            self.c.add(gates.X(instruction_qubit+1)) #rho_B